![alt text](./Cerny_logo_1.jpg)

# Analysis of Cerny ventilation recordings - SIMV-VG versus SIMV

The data processed and analysed in this Notebook were collected by the **Neonatal Emergency and Transport Service of the Peter Cerny Foundation**, Budapest, Hungary

**Author: Dr Gusztav Belteki**

## Analysis of mechanically ventilated cases among `AL000001-AL000300`

Analysis of ventilated babies comparing ventilator parameters during volume targeted ventilation versus non volume targeted ventilation (VTV).

This notebook compares SIMV-VG ventilation with SIMV ventilation without VG.

- It starts from 145 ventilated cases
- 13 cases with >46 week CGA are excluded - 132 remaining
- 48 cases are excluded as it was not SIMV or SIMV was not the dominant mode in them - 84 remaining 
- For 1 case VG info was missing - 83 remaining: 49 VG, 34 no-VG
- 6 cases removed as they contained >15 minutes of the other ventilator mode (VG or no-VG): **77 cases remaining (45 VG, 32 noVG**)

For cases where SIMV was the dominant mode but had other modes, parts of the recording containing modes have been removed.

Imported: 

- data_pars_measurements_ventilated_1_300.pickle,  
- data_pars_settings_ventilated_1_300.pickle, 
- data_pars_alarms_ventilated_1_300.pickle, 
- vent_modes_ventilated_1_300_plus.pickle, 
- clin_df_pickle_1_300.pickle, 
- blood_gases_1_300.pickle,
- Fabian_parameters.xlsx

Exported: 

- Excel files and graphs about statistics regarding clinical data, ventilator modes and ventilator parameters  are exported to **/Users/guszti/ventilation_fabian/Analyses/analysis_ventilated_1_300_SIMV_VG_noVG** folder
- Time series graphs on ventilator parameters are exported to the **/Volumes/GUSZTI/data_dump/fabian/fabian_cases** folder into individual subfolders named after the recording
- Tables and figures of the manuscript


### Importing the necessary libraries and setting options

In [ ]:
import IPython
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk

import os
import sys
import re
import pickle

from scipy import stats
from pandas import Series, DataFrame
from datetime import datetime, timedelta

%matplotlib inline
matplotlib.style.use('classic')
matplotlib.rcParams['figure.facecolor'] = 'w'

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
# pd.set_option('mode.chained_assignment', None) 

In [ ]:
print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("NumPy version: {}".format(np.__version__))
print("SciPy version: {}".format(sp.__version__))
print("IPython version: {}".format(IPython.__version__))
print("scikit-learn version: {}".format(sk.__version__))

### List and set the working directory and the directory to write out data

In [ ]:
# Topic of the Notebook which will also be the name of the subfolder containing results
TOPIC = 'fabian'

# Name of the external hard drive
DRIVE = 'GUSZTI'

# Directory containing clinical and blood gas data
CWD = '/Users/guszti/ventilation_fabian'

# Directory on external drive to read the ventilation data from
DIR_READ = '/Volumes/%s/Fabian/fabian_data' % DRIVE

DIR_WRITE = '%s/%s/%s' % (CWD, 'Analyses', 'analysis_ventilated_1_300_SIMV_VG_noVG')
if not os.path.isdir(DIR_WRITE):
    os.makedirs(DIR_WRITE)

# Images and raw data will be written on an external hard drive
if not os.path.isdir('/Volumes/%s/data_dump/%s' % (DRIVE, TOPIC)):
    os.makedirs('/Volumes/%s/data_dump/%s' % (DRIVE, TOPIC))
DATA_DUMP = '/Volumes/%s/data_dump/%s' % (DRIVE, TOPIC)

In [ ]:
os.chdir(CWD)
os.getcwd()

In [ ]:
DIR_READ

In [ ]:
DIR_WRITE

In [ ]:
DATA_DUMP

### Import pickle archives

In [ ]:
# Import ventilator parameters, settings and alarms

with open('%s/%s.pickle' % (DATA_DUMP, 'clin_df_1_300'), 'rb') as handle:
    clin_df = pickle.load(handle)
    
with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_measurements_ventilated_1_300'), 'rb') as handle:
    data_pars_measurements_ventilated = pickle.load(handle)
    
with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_settings_ventilated_1_300'), 'rb') as handle:
    data_pars_settings_ventilated = pickle.load(handle)
    
with open('%s/%s.pickle' % (DATA_DUMP, 'data_pars_alarms_ventilated_1_300'), 'rb') as handle:
    data_pars_alarms_ventilated = pickle.load(handle)

In [ ]:
### Import blood gases
with open('%s/%s.pickle' % (DATA_DUMP, 'blood_gases_1_300'), 'rb') as handle:
    blood_gases = pickle.load(handle)

In [ ]:
# Import DataFrame with ventilation modes

with open('%s/%s.pickle' % (DATA_DUMP, 'vent_modes_ventilated_1_300_plus'), 'rb') as handle:
    vent_modes_ventilated = pickle.load(handle)

In [ ]:
cases = sorted(data_pars_measurements_ventilated.keys())

In [ ]:
len(data_pars_measurements_ventilated)

### Import table for interpreting ventilator parameters

In [ ]:
par_key_table = pd.read_excel('Fabian_parameters.xlsx')
par_key_table;

### Exclude those cases whose corrected age was over Term + 6 weeks

In [ ]:
cases_old = [case for case in cases if clin_df.loc[case]['Corrected gestational Age (weeks)'] > 46]
len(cases_old)

In [ ]:
cases = [case for case in cases if clin_df.loc[case]['Corrected gestational Age (weeks)'] <= 46]
len(cases)

In [ ]:
vent_modes_ventilated = vent_modes_ventilated.loc[cases]
len(vent_modes_ventilated)

### Consider only cases that are completely or predominantly SIMV ventilation

In [ ]:
vent_modes_ventilated_has_simv = vent_modes_ventilated[vent_modes_ventilated[['SIMV']].sum(axis = 1) > 0]
vent_modes_ventilated_has_simv.sort_index(inplace = True)
cases = sorted(vent_modes_ventilated_has_simv.index)                                         

In [ ]:
vent_modes_ventilated_has_simv.head()

In [ ]:
len(vent_modes_ventilated_has_simv), len(cases)

In [ ]:
# How many has multiple modes
has_multiple_modes = vent_modes_ventilated_has_simv[vent_modes_ventilated_has_simv['multiple_mode'] == 'Yes']
len(has_multiple_modes)

In [ ]:
has_multiple_modes

In [ ]:
# Which are the ones where SIMV / SIMV-PSV are predominant modes
a = has_multiple_modes
simv_dominant = a[(a['SIMV'] / a['total']) > 0.5]
simv_dominant

In [ ]:
len(simv_dominant)

In [ ]:
a = has_multiple_modes
simv_not_dominant = a[(a['SIMV'] / a['total']) <= 0.5]
simv_not_dominant

### Remove those recordings completely where the dominant mode is not simv

In [ ]:
cases_simv = [case for case in cases if case not in simv_not_dominant.index]
vent_modes_simv = vent_modes_ventilated_has_simv.loc[cases_simv]
vent_modes_simv['VG_off'] = vent_modes_simv['total'] - vent_modes_simv['VG_on']
len(cases_simv), len(vent_modes_simv)

### Remove recording(s) where VG info is not available

In [ ]:
vent_modes_simv.dropna(how = 'all', subset = ['VG_on'], inplace = True)

cases_simv = [case for case in cases if case in vent_modes_simv.index]
len(cases_simv), len(vent_modes_simv)

### For the mixed recordings remove those parts that are not SIMV

In [ ]:
before = {}
for recording in sorted(data_pars_measurements_ventilated.keys()):
    before[recording] = len(data_pars_measurements_ventilated[recording])

In [ ]:
for case in sorted(simv_dominant.index):
    a = data_pars_settings_ventilated[case]
    data_pars_settings_ventilated[case] = a[a['Ventilator_mode'] == 'SIMV']
    data_pars_measurements_ventilated[case] = \
        data_pars_measurements_ventilated[case].reindex(index = data_pars_settings_ventilated[case].index)
    data_pars_alarms_ventilated[case] = \
        data_pars_alarms_ventilated[case].reindex(index = data_pars_settings_ventilated[case].index)

In [ ]:
after = {}
for recording in sorted(data_pars_measurements_ventilated.keys()):
    after[recording] = len(data_pars_measurements_ventilated[recording])

In [ ]:
# how much data was removed?
for recording in sorted(before.keys()):
    if (before[recording] - after[recording]) > 0:
        print(recording, 'removed %d seconds' % (2 * (before[recording] - after[recording])))

### Identify cases who received mechanical ventilation with or without VG

In [ ]:
len(vent_modes_simv)

In [ ]:
vent_modes_simv.head()

In [ ]:
vent_modes_simv_vg = vent_modes_simv[vent_modes_simv['VG_on'] > vent_modes_simv['VG_off']]
cases_simv_vg = sorted(vent_modes_simv_vg.index)
len(cases_simv_vg)

In [ ]:
vent_modes_simv_no_vg = vent_modes_simv[vent_modes_simv['VG_on'] < vent_modes_simv['VG_off']]
cases_simv_no_vg = sorted(vent_modes_simv_no_vg.index)
len(cases_simv_no_vg)

### Remove those recordings that have more than 15 minutes (900 seconds) of the other ventilator mode

In [ ]:
cases_simv_vg = [case for case in cases_simv_vg if vent_modes_simv_vg.loc[case]['VG_off'] < 900]
vent_modes_simv_vg = vent_modes_simv_vg.loc[cases_simv_vg]
len(cases_simv_vg), len(vent_modes_simv_vg.index)

In [ ]:
cases_simv_no_vg = [case for case in cases_simv_no_vg if vent_modes_simv_no_vg.loc[case]['VG_on'] < 900]
vent_modes_simv_no_vg = vent_modes_simv_no_vg.loc[cases_simv_no_vg]
len(cases_simv_no_vg), len(vent_modes_simv_no_vg.index)

In [ ]:
vent_modes_simv;

### Remove non-VG parts of VG recordings and vice versa

In [ ]:
data_pars_settings_vg = {}
data_pars_measurements_vg = {}
data_pars_alarms_vg = {}

for case in cases_simv_vg:
    a = data_pars_settings_ventilated[case]
    data_pars_settings_vg[case] = a[a['VG_state'] == 'on'].copy()
    data_pars_measurements_vg[case] = \
        data_pars_measurements_ventilated[case].reindex(index = data_pars_settings_vg[case].index)
    data_pars_alarms_vg[case] = \
        data_pars_alarms_ventilated[case].reindex(index = data_pars_settings_vg[case].index)

In [ ]:
data_pars_settings_no_vg = {}
data_pars_measurements_no_vg = {}
data_pars_alarms_no_vg = {}

for case in cases_simv_no_vg:

    try:
        a = data_pars_settings_ventilated[case]
        data_pars_settings_no_vg[case] = a[a['VG_state'] != 'on']
        
    except KeyError:
        data_pars_settings_no_vg[case] = \
            data_pars_settings_ventilated[case].copy()
            
   
    data_pars_measurements_no_vg[case] = \
        data_pars_measurements_ventilated[case].reindex(index = data_pars_settings_no_vg[case].index)
    data_pars_alarms_no_vg[case] = \
        data_pars_alarms_ventilated[case].reindex(index = data_pars_settings_no_vg[case].index)

In [ ]:
len(data_pars_settings_vg)

In [ ]:
len(data_pars_settings_no_vg)

In [ ]:
# How many rows (= 2 sec each) were removed
for case in cases_simv_vg:
    if len(data_pars_measurements_ventilated[case]) - len(data_pars_measurements_vg[case]) > 0:
        print(case, '%d seconds were removed' % (2 * (len(data_pars_measurements_ventilated[case]) -\
           len(data_pars_measurements_vg[case]))))

In [ ]:
# How many rows (= 2 sec each) were removed
for case in cases_simv_no_vg:
    if len(data_pars_measurements_ventilated[case]) - len(data_pars_measurements_no_vg[case]) > 0:
        print(case, '%d seconds were removed' % (2* (len(data_pars_measurements_ventilated[case]) -\
           len(data_pars_measurements_no_vg[case]))))

### Duration of the final set of SIMV-VG and SIMV-no-VG recordings

In [ ]:
recording_durations_vg = {}
for rec in data_pars_measurements_vg:
    # As sampling rate was 0.5 Hz, the length is seconds is the number or rows * 2
    recording_durations_vg[rec] = len(data_pars_measurements_vg[rec]) * 2

recording_durations_vg = DataFrame(recording_durations_vg, index = ['recording_duration']).T
    
recording_durations_no_vg = {}
for rec in data_pars_measurements_no_vg:
    # As sampling rate was 0.5 Hz, the length is seconds is the number or rows * 2
    recording_durations_no_vg[rec] = len(data_pars_measurements_no_vg[rec]) * 2

recording_durations_no_vg = DataFrame(recording_durations_no_vg, index = ['recording_duration']).T

In [ ]:
recording_durations_vg.sort_values(by = 'recording_duration');

In [ ]:
recording_durations_no_vg.sort_values(by = 'recording_duration');

In [ ]:
recording_durations_vg['recording_duration'].describe() / 60

In [ ]:
recording_durations_vg['recording_duration'].sum(), recording_durations_vg['recording_duration'].sum()/60

In [ ]:
recording_durations_no_vg['recording_duration'].describe() / 60

In [ ]:
recording_durations_no_vg['recording_duration'].sum(), recording_durations_no_vg['recording_duration'].sum()/ 60

In [ ]:
stats.mannwhitneyu(recording_durations_vg['recording_duration'], recording_durations_no_vg['recording_duration'])

### Comparative statistics on clinical details between VG and no VG

In [ ]:
clin_df_VG = clin_df.loc[cases_simv_vg]
clin_df_no_VG = clin_df.loc[cases_simv_no_vg]

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG' ]

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([clin_df_VG['Gestational Age (weeks)'] , 
             clin_df_no_VG['Gestational Age (weeks)'] ], widths = 0.5, 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_ylim(0, 48)
ax.set_xlabel('ventilation mode', size = 14)
ax.set_ylabel('weeks', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
ax.set_xticklabels(xticklabels)
ax.set_title('Gestational age')
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'ventilated_gest_age_VG_no_VG', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG' ]

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([clin_df_VG['Corrected gestational Age (weeks)'] , 
             clin_df_no_VG['Corrected gestational Age (weeks)']], widths = 0.5, 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_ylim(0, 58)
ax.set_xlabel('ventilation mode', size = 14)
ax.set_ylabel('weeks', size = 14)
ax.set_title('Corrected gestational age')
ax.tick_params(axis='both', which='major', labelsize=14)
ax.set_xticklabels(xticklabels)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'ventilated_corr_gest_age_VG_no_VG', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG' ]

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([clin_df_VG['Birth Weight'], 
             clin_df_no_VG['Birth Weight']], widths = 0.5, 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)
ax.set_ylim(0, 5800)
ax.set_xlabel('ventilation mode', size = 14)
ax.set_ylabel('birth weight (grams)', size = 14)
ax.set_title('Birth weight')
ax.tick_params(axis='both', which='major', labelsize=14)
ax.set_xticklabels(xticklabels)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'ventilated_birth_weight_VG_no_VG', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG' ]

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([clin_df_VG['Weight'],
             clin_df_no_VG['Weight']], widths = 0.5, 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_ylim(0, 5800)
ax.set_xlabel('ventilation mode', size = 14)
ax.set_ylabel('actual weight (grams)', size = 14)
ax.set_title('Actual weight')
ax.tick_params(axis='both', which='major', labelsize=14)
ax.set_xticklabels(xticklabels)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'ventilated_actual_weight_VG_no_VG', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

### Comparative statistics between clinical details, VG versus no VG

##### Using parametric statistics

In [ ]:
pars = ['Gestational Age (weeks)', 'Corrected gestational Age (weeks)', 'Birth Weight', 'Weight']

clin_df_stats_VG = round(DataFrame([clin_df_VG[pars].mean(), 
                                    clin_df_VG[pars].std()]).T, 2)
clin_df_stats_VG.columns = ['mean', 'SD']

clin_df_stats_no_VG = round(DataFrame([clin_df_no_VG[pars].mean(), 
                                       clin_df_no_VG[pars].std()]).T, 2)
clin_df_stats_no_VG.columns = ['mean', 'SD']

clin_df_stats_VG_no_VG = pd.concat([clin_df_stats_VG, clin_df_stats_no_VG], axis = 1,
                              keys = ['VG', 'no VG'])

In [ ]:
ttest_stat = []
ttest_p = []
for par in pars:
    ttest_stat.append(stats.ttest_ind(clin_df_VG[par], clin_df_no_VG[par])[0])
    ttest_p.append(stats.ttest_ind(clin_df_VG[par],  clin_df_no_VG[par])[1])

clin_df_stats_VG_no_VG['p'] = ttest_p  

In [ ]:
clin_df_stats_VG_no_VG

##### Using non-parametric statistics

In [ ]:
pars = ['Gestational Age (weeks)', 'Corrected gestational Age (weeks)', 'Birth Weight', 'Weight']

clin_df_stats_VG_nonpar = round(DataFrame([clin_df_VG[pars].median(), 
                                    clin_df_VG[pars].apply(lambda x: np.percentile(x, 25)),
                                    clin_df_VG[pars].apply(lambda x: np.percentile(x, 75))]).T, 2)
clin_df_stats_VG_nonpar.columns = ['median', '25pc', '75pc']

clin_df_stats_no_VG_nonpar = round(DataFrame([clin_df_no_VG[pars].median(), 
                                    clin_df_no_VG[pars].apply(lambda x: np.percentile(x, 25)),
                                    clin_df_no_VG[pars].apply(lambda x: np.percentile(x, 75))]).T, 2)
clin_df_stats_no_VG_nonpar.columns = ['median', '25pc', '75pc']

clin_df_stats_VG_no_VG_nonpar = pd.concat([clin_df_stats_VG_nonpar, clin_df_stats_no_VG_nonpar], axis = 1,
                              keys = ['VG', 'no VG'])

In [ ]:
ttest_stat_nonpar = []
ttest_p_nonpar = []
for par in pars:
    ttest_stat_nonpar.append(stats.mannwhitneyu(clin_df_VG[par], clin_df_no_VG[par])[0])
    ttest_p_nonpar.append(stats.mannwhitneyu(clin_df_VG[par],  clin_df_no_VG[par])[1])

clin_df_stats_VG_no_VG_nonpar['p'] = ttest_p_nonpar 

In [ ]:
clin_df_stats_VG_no_VG_nonpar

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'clin_df_stats_VG_no_VG.xlsx'))
clin_df_stats_VG_no_VG.to_excel(writer, 'VG_no_VG')
clin_df_stats_VG_no_VG_nonpar.to_excel(writer, 'VG_no_VG_nonpar')
writer.save()

In [ ]:
clin_df_VG;

In [ ]:
clin_df_no_VG;

### Calculate respiratory severity score for the first and last five minutes of the transport

In [ ]:
# 5 minutes = 150 data points with 0.5 Hz sampling rate

rss_start_vg = {}

for case in cases_simv_vg:
    rss_start_vg[case] = round((data_pars_settings_vg[case]['FiO2_set'][0:150] * 
                              data_pars_measurements_vg[case]['MAP'][0:150] / 100).mean(), 2)
    
rss_end_vg = {}

for case in cases_simv_vg:
    rss_end_vg[case] = round((data_pars_settings_vg[case]['FiO2_set'][-150:] * 
                              data_pars_measurements_vg[case]['MAP'][-150:] / 100).mean(), 2)
    
rss_vg_frame = DataFrame([rss_start_vg, rss_end_vg]).T
rss_vg_frame.columns = ['start', 'end']

In [ ]:
rss_start_no_vg = {}

for case in cases_simv_no_vg:
    rss_start_no_vg[case] = round((data_pars_settings_no_vg[case]['FiO2_set'][0:150] * 
                              data_pars_measurements_no_vg[case]['MAP'][0:150] / 100).mean(), 2)
rss_end_no_vg = {}

for case in cases_simv_no_vg:
    rss_end_no_vg[case] = round((data_pars_settings_no_vg[case]['FiO2_set'][-150:] * 
                              data_pars_measurements_no_vg[case]['MAP'][-150:] / 100).mean(), 2)
    
rss_no_vg_frame = DataFrame([rss_start_no_vg, rss_end_no_vg]).T
rss_no_vg_frame.columns = ['start', 'end']

In [ ]:
rss_vg_frame.head()

In [ ]:
rss_no_vg_frame.head()

In [ ]:
round(rss_vg_frame.describe(), 1)

In [ ]:
round(rss_no_vg_frame.describe(), 1)

In [ ]:
stats.mannwhitneyu(rss_vg_frame['start'], rss_no_vg_frame['start'])

In [ ]:
stats.mannwhitneyu(rss_vg_frame['end'], rss_no_vg_frame['end'])

In [ ]:
stats.wilcoxon(rss_vg_frame['start'], rss_vg_frame['end'])

In [ ]:
stats.wilcoxon(rss_no_vg_frame['start'], rss_no_vg_frame['end'])

## Compare the aggregate statistics `mean` and `sd` of the recordings between the VG and no VG groups

### Calculate descriptive statistics for selected ventilator parameters and settings in each recording

##### RR data are missing in SIMV cases !!

This is because for SIMV recordings the `RR` values are 0 or missing. There is, however `RR`reading for SIPPV and SIMVPSV recordings. See the script below to test it.

`for key in sorted(data_pars_measurements_ventilated.keys()):
    try:
        print(key, '   ', data_pars_measurements_ventilated[key]['RR'].unique())
    except KeyError:
        print('No data for %s' % key)`

###### `VTemand_pat` has been wrongly labeled. This is the VT of the spontaneous breaths. Name should be `VEspon_pat`

In [ ]:
selected_parameters = ['Leak', 'MAP', 'MV_kg', 'MVresp', 'PEEP', 'PIP', 'VTimand_kg', 'VTemand_kg', 
                       'VTespon_pat_kg', 'VTemand_resp_kg',  ]

stats_pars_measurements_vg = {} 
for case in cases_simv_vg:
    stats_pars_measurements_vg[case] = \
        round(data_pars_measurements_vg[case][selected_parameters].describe(percentiles = 
                                                                            (0.05, 0.25, 0.5, 0.75, 0.95)), 2)
    stats_pars_measurements_vg[case].index = ['data_points', 'mean', 'SD', 'min', '5pc', 
                                     '25pc', 'median', '75pc', '95pc', 'max']
    
stats_pars_measurements_no_vg = {} 
for case in cases_simv_no_vg:
    stats_pars_measurements_no_vg[case] = \
        round(data_pars_measurements_no_vg[case][selected_parameters].describe(percentiles = 
                                                                               (0.05, 0.25, 0.5, 0.75, 0.95)), 2)
    stats_pars_measurements_no_vg[case].index = ['data_points', 'mean', 'SD', 'min', '5pc', 
                                     '25pc', 'median', '75pc', '95pc', 'max']

In [ ]:
stats_pars_measurements_vg_frame = pd.concat(stats_pars_measurements_vg)
stats_pars_measurements_no_vg_frame = pd.concat(stats_pars_measurements_no_vg)

In [ ]:
selected_settings_vg = ['FiO2_set', 'PIP_set', 'PEEP_set', 'VG_set_kg', 'RR_set', 'Ti_set']
selected_settings_no_vg = ['FiO2_set', 'PIP_set', 'PEEP_set', 'RR_set', 'Ti_set']

stats_pars_settings_vg = {} 
for case in cases_simv_vg:
    stats_pars_settings_vg[case] = round(data_pars_settings_vg[case][selected_settings_vg].describe(percentiles 
                                                                         = (0.05, 0.25, 0.5, 0.75, 0.95)), 2)
    stats_pars_settings_vg[case].index = ['data_points', 'mean', 'SD', 'min', '5pc', 
                                     '25pc', 'median', '75pc', '95pc', 'max']

stats_pars_settings_no_vg = {} 
for case in cases_simv_no_vg:
    stats_pars_settings_no_vg[case] = round(data_pars_settings_no_vg[case][selected_settings_no_vg].describe(percentiles
                                                                        = (0.05, 0.25, 0.5, 0.75, 0.95)), 2)
    stats_pars_settings_no_vg[case].index = ['data_points', 'mean', 'SD', 'min', '5pc', 
                                     '25pc', 'median', '75pc', '95pc', 'max']

In [ ]:
stats_pars_settings_vg_frame = pd.concat(stats_pars_settings_vg)
stats_pars_settings_no_vg_frame = pd.concat(stats_pars_settings_no_vg)

In [ ]:
stats_pars_all_vg_frame = pd.concat([stats_pars_measurements_vg_frame, stats_pars_settings_vg_frame], axis = 1)
stats_pars_all_no_vg_frame = pd.concat([stats_pars_measurements_no_vg_frame, stats_pars_settings_no_vg_frame], axis = 1)

stats_pars_all_vg_frame_2 = stats_pars_all_vg_frame.unstack()
stats_pars_all_no_vg_frame_2 = stats_pars_all_no_vg_frame.unstack()

In [ ]:
stats_pars_all_vg_frame.head()

In [ ]:
stats_pars_all_vg_frame_2.head()

In [ ]:
stats_pars_all_no_vg_frame.head()

In [ ]:
stats_pars_all_no_vg_frame_2.head()

In [ ]:
stats_pars_all_vg_frame_2['Leak'].sort_values('mean')

In [ ]:
stats_pars_all_vg_frame_2['Leak']['mean'].describe()

In [ ]:
stats_pars_all_no_vg_frame_2['Leak'].sort_values('mean')

In [ ]:
stats_pars_all_no_vg_frame_2['Leak']['mean'].describe()

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_leak.xlsx'))
stats_pars_all_vg_frame_2['Leak'].sort_values('mean').to_excel(writer, 'SIMV_VG')
stats_pars_all_no_vg_frame_2['Leak'].sort_values('mean').to_excel(writer, 'SIMV_noVG')
writer.save()

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_all.xlsx'))
stats_pars_all_vg_frame.to_excel(writer, 'SIMV_VG_parameters_1')
stats_pars_all_vg_frame_2.to_excel(writer, 'SIMV_VG_parameters_2')
stats_pars_all_no_vg_frame.to_excel(writer, 'SIMV_no_VG_parameters_1')
stats_pars_all_no_vg_frame_2.to_excel(writer, 'SIMV_no_VG_parameters_2')
writer.save()

### The `arithmetic mean` of the selected ventilator parameters and settings in the individual recordings

In [ ]:
vg_mean = {}
for case in cases_simv_vg:
    vg_mean[case] = stats_pars_measurements_vg[case].loc['mean']
vg_mean_frame = DataFrame(vg_mean)

no_vg_mean = {}
for case in cases_simv_no_vg:
    no_vg_mean[case] = stats_pars_measurements_no_vg[case].loc['mean']
no_vg_mean_frame = DataFrame(no_vg_mean)

In [ ]:
vg_mean_2 = {}
for case in cases_simv_vg:
    vg_mean_2[case] = stats_pars_settings_vg[case].loc['mean']
vg_mean_frame_2 = DataFrame(vg_mean_2)

no_vg_mean_2 = {}
for case in cases_simv_no_vg:
    no_vg_mean_2[case] = stats_pars_settings_no_vg[case].loc['mean']
no_vg_mean_frame_2 = DataFrame(no_vg_mean_2)

In [ ]:
vg_mean_frame = pd.concat([vg_mean_frame, vg_mean_frame_2])
no_vg_mean_frame = pd.concat([no_vg_mean_frame, no_vg_mean_frame_2])

In [ ]:
vg_mean_frame.T.head()

In [ ]:
no_vg_mean_frame.T.head()

### `SDs` of the various ventilator parameters in the individual recordings

In [ ]:
vg_sd = {}
for case in cases_simv_vg:
    vg_sd[case] = stats_pars_measurements_vg[case].loc['SD']
vg_sd_frame = DataFrame(vg_sd)

no_vg_sd = {}
for case in cases_simv_no_vg:
    no_vg_sd[case] = stats_pars_measurements_no_vg[case].loc['SD']
no_vg_sd_frame = DataFrame(no_vg_sd)

In [ ]:
vg_sd_2 = {}
for case in cases_simv_vg:
    vg_sd_2[case] = stats_pars_settings_vg[case].loc['SD']
vg_sd_frame_2 = DataFrame(vg_sd_2)

no_vg_sd_2 = {}
for case in cases_simv_no_vg:
    no_vg_sd_2[case] = stats_pars_settings_no_vg[case].loc['SD']
no_vg_sd_frame_2 = DataFrame(no_vg_sd_2)

In [ ]:
vg_sd_frame = pd.concat([vg_sd_frame, vg_sd_frame_2])
no_vg_sd_frame = pd.concat([no_vg_sd_frame, no_vg_sd_frame_2])

In [ ]:
vg_sd_frame.T.head()

In [ ]:
no_vg_sd_frame.T.head()

### The `median` of the selected ventilator parameters and settings in the individual recordings

In [ ]:
vg_median = {}
for case in cases_simv_vg:
    vg_median[case] = stats_pars_measurements_vg[case].loc['median']
vg_median_frame = DataFrame(vg_median)

no_vg_median = {}
for case in cases_simv_no_vg:
    no_vg_median[case] = stats_pars_measurements_no_vg[case].loc['median']
no_vg_median_frame = DataFrame(no_vg_median)

In [ ]:
vg_median_2 = {}
for case in cases_simv_vg:
    vg_median_2[case] = stats_pars_settings_vg[case].loc['median']
vg_median_frame_2 = DataFrame(vg_median_2)

no_vg_median_2 = {}
for case in cases_simv_no_vg:
    no_vg_median_2[case] = stats_pars_settings_no_vg[case].loc['median']
no_vg_median_frame_2 = DataFrame(no_vg_median_2)

In [ ]:
vg_median_frame = pd.concat([vg_median_frame, vg_median_frame_2])
no_vg_median_frame = pd.concat([no_vg_median_frame, no_vg_median_frame_2])

In [ ]:
vg_median_frame.T.head()

In [ ]:
no_vg_median_frame.T.head()

### `SDs` of the various ventilator parameters in the individual recordings

In [ ]:
vg_sd = {}
for case in cases_simv_vg:
    vg_sd[case] = stats_pars_measurements_vg[case].loc['SD']
vg_sd_frame = DataFrame(vg_sd)

no_vg_sd = {}
for case in cases_simv_no_vg:
    no_vg_sd[case] = stats_pars_measurements_no_vg[case].loc['SD']
no_vg_sd_frame = DataFrame(no_vg_sd)

In [ ]:
vg_sd_2 = {}
for case in cases_simv_vg:
    vg_sd_2[case] = stats_pars_settings_vg[case].loc['SD']
vg_sd_frame_2 = DataFrame(vg_sd_2)

no_vg_sd_2 = {}
for case in cases_simv_no_vg:
    no_vg_sd_2[case] = stats_pars_settings_no_vg[case].loc['SD']
no_vg_sd_frame_2 = DataFrame(no_vg_sd_2)

In [ ]:
vg_sd_frame = pd.concat([vg_sd_frame, vg_sd_frame_2])
no_vg_sd_frame = pd.concat([no_vg_sd_frame, no_vg_sd_frame_2])

In [ ]:
vg_sd_frame.T.head()

In [ ]:
no_vg_sd_frame.T.head()

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_vg_no_vg.xlsx'))
vg_mean_frame.to_excel(writer, 'vg_mean_frame')
no_vg_mean_frame.to_excel(writer, 'no_vg_mean_frame')
vg_sd_frame.to_excel(writer, 'vg_sd_frame')
no_vg_sd_frame.to_excel(writer, 'no_vg_sd_frame')
writer.save()

### Inferential statistics of comparing ` group medians`  of `mean ventilator parameters` between  VG and no VG groups

In [ ]:
def iqr(ser):
    return round(ser.quantile(0.25), 2), round(ser.quantile(0.75), 2)  

In [ ]:
def minmax(ser):
    return round(ser.min(), 2), round(ser.max(), 2)

In [ ]:
pars = ['FiO2_set', 'PEEP_set','RR_set', 'Ti_set',  'PIP', 'MAP', 'PEEP', 'MV_kg', 'MVresp', 
        'VTimand_kg', 'VTespon_pat_kg', 'VTemand_resp_kg', 'Leak', ]

group_median_vg_mean = []
group_iqr_vg_mean = []
group_range_vg_mean = []
group_median_no_vg_mean = []
group_iqr_no_vg_mean = []
group_range_no_vg_mean = []
mw_test_mean = []

for par in pars:
    group_median_vg_mean.append(vg_mean_frame.loc[par].median())
    group_iqr_vg_mean.append(iqr(vg_mean_frame.loc[par]))
    group_range_vg_mean.append(minmax(vg_mean_frame.loc[par]))
    
for par in pars:
    group_median_no_vg_mean.append(no_vg_mean_frame.loc[par].median())
    group_iqr_no_vg_mean.append(iqr(no_vg_mean_frame.loc[par]))
    group_range_no_vg_mean.append(minmax(no_vg_mean_frame.loc[par]))
    
    mw_test_mean.append(stats.mannwhitneyu(vg_mean_frame.loc[par].dropna(), 
                                              no_vg_mean_frame.loc[par].dropna())[1])

VG_stats_mean = round(DataFrame([group_median_vg_mean, group_median_no_vg_mean, group_iqr_vg_mean, 
                     group_iqr_no_vg_mean, group_range_vg_mean, group_range_no_vg_mean]).T, 2)

VG_stats_mean.columns = ['VG_mean (group median)', 'no_VG_mean (group median)',
                         'VG_mean (group iqr)', 'no_VG_mean (group iqr)',
                         'VG_mean (group range)', 'no_VG_mean (group range)']

VG_stats_mean['p value (means)'] = mw_test_mean

VG_stats_mean.index = pars
VG_stats_mean.sort_values(by = 'p value (means)', inplace = True)

In [ ]:
VG_stats_mean

In [ ]:
# Target tidal volume
vg_mean_frame.loc['VG_set_kg'].median(), iqr(vg_mean_frame.loc['VG_set_kg'])

In [ ]:
# Pmax during VG
vg_mean_frame.loc['PIP_set'].median(), iqr(vg_mean_frame.loc['PIP_set'])

In [ ]:
# Set PIP during non-VG
no_vg_mean_frame.loc['PIP_set'].median(), iqr(no_vg_mean_frame.loc['PIP_set'])

### Inferential statistics of comparing ` group medians`  of `SDs` of ventilator parameters` between  VG and no VG groups

In [ ]:
pars = ['FiO2_set', 'PEEP_set','RR_set', 'Ti_set',  'PIP', 'MAP', 'PEEP', 'MV_kg', 'MVresp', 
        'VTimand_kg', 'VTespon_pat_kg', 'VTemand_resp_kg', 'Leak', ]

group_median_vg_sd = []
group_iqr_vg_sd = []
group_range_vg_sd = []
group_median_no_vg_sd = []
group_iqr_no_vg_sd = []
group_range_no_vg_sd = []
mw_test_sd = []

for par in pars:
    group_median_vg_sd.append(vg_sd_frame.loc[par].median())
    group_iqr_vg_sd.append(iqr(vg_sd_frame.loc[par]))
    group_range_vg_sd.append(minmax(vg_sd_frame.loc[par]))
    group_median_no_vg_sd.append(no_vg_sd_frame.loc[par].median())
    group_iqr_no_vg_sd.append(iqr(no_vg_sd_frame.loc[par]))
    group_range_no_vg_sd.append(minmax(no_vg_sd_frame.loc[par]))
    
    mw_test_sd.append(stats.mannwhitneyu(vg_sd_frame.loc[par].dropna(),  no_vg_sd_frame.loc[par].dropna())[1])

VG_stats_sd = round(DataFrame([group_median_vg_sd, group_median_no_vg_sd, group_iqr_vg_sd, 
                     group_iqr_no_vg_sd, group_range_vg_sd, group_range_no_vg_sd]).T,  2)

VG_stats_sd.columns = ['VG_sd (group median)', 'no_VG_sd (group median)',
                       'VG_sd (group iqr)', 'no_VG_sd (group iqr)',
                       'VG_sd (group range)', 'no_VG_sd (group range)']

VG_stats_sd['p value (SDs)'] = mw_test_sd

VG_stats_sd.index = pars
VG_stats_sd.sort_values(by = 'p value (SDs)', inplace = True)

In [ ]:
VG_stats_sd

### Correction for multiple testing using false discovery rate (Benjamini-Hochberg)

One good technique for controlling the false discovery rate was briefly mentioned by Simes (1986) and developed in detail by Benjamini and Hochberg (1995). Put the individual P values in order, from smallest to largest. The smallest P value has a rank of i=1, then next smallest has i=2, etc. Compare each individual P value to its Benjamini- Hochberg critical value, (i/m)Q, where i is the rank, m is the total number of tests, and Q is the false discovery rate you choose. The largest P value that has P<(i/m)Q is significant, and all of the P values smaller than it are also significant, even the ones that aren’t less than their Benjamini-Hochberg critical value.

In [ ]:
def fdr(ser, rate = 0.05):
    '''
    
    '''
    result = []
    m = len(ser)
    ser_sorted = ser.sort_values()
    for i in range(m):
        result.append(   round(((i+1) / m * rate), 3))

    return result

In [ ]:
p_values_mean = VG_stats_mean[['p value (means)']].copy()
p_values_mean['stat'] = 'mean'
p_values_mean.columns = ['p value', 'stat']
p_values_sd = VG_stats_sd[['p value (SDs)']].copy()
p_values_sd['stat'] = 'sd'
p_values_sd.columns = ['p value', 'stat']
p_values_combined = pd.concat([p_values_mean, p_values_sd]).sort_values('p value')
p_values_combined = DataFrame(p_values_combined)
p_values_combined = p_values_combined[['stat', 'p value']]

In [ ]:
p_values_combined;

In [ ]:
p_values_combined_fdr = fdr(p_values_combined['p value'])
p_values_combined['fdr'] =  p_values_combined_fdr
p_values_combined['corrected p'] = p_values_combined['p value'] * (0.05 / p_values_combined['fdr'])

In [ ]:
# p values < 0.023 were statistically significant
p_values_combined

In [ ]:
p_values_combined_mean = p_values_combined[p_values_combined['stat'] == 'mean']
VG_stats_mean = VG_stats_mean.join(p_values_combined_mean)
VG_stats_mean

In [ ]:
p_values_combined_sd = p_values_combined[p_values_combined['stat'] == 'sd']
VG_stats_sd = VG_stats_sd.join(p_values_combined_sd)
VG_stats_sd

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'inferential_stats.xlsx'))
VG_stats_mean.to_excel(writer, 'VG_stats_mean')
VG_stats_sd.to_excel(writer, 'VG_stats_sd')
writer.save()

## Analysing the `mean` ventilation parameters of the recordings and comparing the two groups

### VG_set_kg

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot(vg_mean_frame.loc['VG_set_kg'], 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 12)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('Mean set VT (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_set_kg', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = [3,4])

y = vg_mean_frame.loc['VG_set_kg']
x = 1.25 + 0.5 * np.random.random(len(y))

ax.scatter(x,y, marker = 'o', color = 'black', s = 5)
ax.set_xlim(0, 3)
ax.set_ylim(0, 10)
ax.set_title('', size = 16)
ax.set_xlabel('VG', size = 14)
ax.set_xticklabels('')
ax.set_ylabel('set VT (mL/kg)', size = 14)
ax.grid(True)
ax.tick_params(axis='both', which='major', labelsize=14)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_set_kg_dotplot', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
bins = np.arange(2, 9.1)

cats_VG_set_kg = pd.cut(vg_mean_frame.loc['VG_set_kg'], bins = bins, right = True)
cats_VG_set_kg.value_counts().sort_index()

In [ ]:
fig, ax = plt.subplots(figsize = [6, 4])
cats_VG_set_kg.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'VG_set_kg', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of recordings', size = 14)
plt.title('Set VT', fontsize = 16)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_set_kg_hist', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
vg_mean_frame.loc['VG_set_kg'].describe()

In [ ]:
# Large set tidal volumes

VG_set_over_6_VG = vg_mean_frame.loc['VG_set_kg'][vg_mean_frame.loc['VG_set_kg'] > 6].sort_values()
VG_set_over_6_VG

In [ ]:
# Low set tidal volumes

VG_set_below_4_VG = vg_mean_frame.loc['VG_set_kg'][vg_mean_frame.loc['VG_set_kg'] < 4].sort_values()
VG_set_below_4_VG

### VTemand_resp_kg

The ventilator inflations whether synchronized or not. It is actually `VTemand_kg`

#### Comparing the mean VTemand_resp of the recordings

In [ ]:
fig, ax = plt.subplots(figsize = (5, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([vg_mean_frame.loc['VTemand_resp_kg'].dropna(), no_vg_mean_frame.loc['VTemand_resp_kg'].dropna()], 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 16)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('Mean VTemand (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_noVG_VTemand_resp', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
bins = np.arange(1, 8.1)

cats_VTemand_vg = pd.cut(vg_mean_frame.loc['VTemand_resp_kg'], bins = bins, right = True)
cats_VTemand_vg.value_counts().sort_index()

In [ ]:
fig, ax = plt.subplots(figsize = [6, 4])
cats_VTemand_vg.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                     color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of recordings', size = 14)
ax.set_ylim(0, 28)
plt.title('VTemand_resp (VG recordings)', fontsize = 16)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'cats_VTemand_vg_hist', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
bins = np.arange(1, 10.1)

cats_VTemand_no_vg = pd.cut(no_vg_mean_frame.loc['VTemand_resp_kg'], bins = bins, right = True)
cats_VTemand_no_vg.value_counts().sort_index()

In [ ]:
fig, ax = plt.subplots(figsize = [6, 4])
cats_VTemand_no_vg.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of recordings', size = 14)
ax.set_ylim(0, 10)
plt.title('VTemand_resp (non-VG recordings)', fontsize = 16)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'cats_VTemand_vg_hist', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
vg_mean_frame.loc['VTemand_resp_kg'].describe()

In [ ]:
no_vg_mean_frame.loc['VTemand_resp_kg'].describe()

##### Over 6 mL/kg

In [ ]:
# High VTemand_resp values

VT_over_6_VG = vg_mean_frame.loc['VTemand_resp_kg'][vg_mean_frame.loc['VTemand_resp_kg'] > 6].sort_values()
VT_over_6_VG

In [ ]:
len(VT_over_6_VG)

In [ ]:
len(VT_over_6_VG) / len(vg_mean_frame.columns)

In [ ]:
VT_over_6_no_VG = no_vg_mean_frame.loc['VTemand_resp_kg'][no_vg_mean_frame.loc['VTemand_resp_kg'] > 6].sort_values()
VT_over_6_no_VG

In [ ]:
len(VT_over_6_no_VG)

#### Over 8 mL/kg

In [ ]:
VT_over_8_no_VG = no_vg_mean_frame.loc['VTemand_resp_kg'][no_vg_mean_frame.loc['VTemand_resp_kg'] > 8].sort_values()
VT_over_8_no_VG

In [ ]:
len(VT_over_8_no_VG)

In [ ]:
len(VT_over_8_no_VG) / len(no_vg_mean_frame.columns)

In [ ]:
no_vg_mean_frame[VT_over_8_no_VG.index]

In [ ]:
no_vg_mean_frame[VT_over_8_no_VG.index].loc['PIP'].median(), no_vg_mean_frame[VT_over_8_no_VG.index].loc['PIP'].min(), \
no_vg_mean_frame[VT_over_8_no_VG.index].loc['PIP'].max()

#### Below 4 mL/kg

In [ ]:
VT_below_4_VG  = vg_mean_frame.loc['VTemand_resp_kg'][vg_mean_frame.loc['VTemand_resp_kg'] < 4].sort_values()
VT_below_4_VG

In [ ]:
VT_below_4_no_VG  = no_vg_mean_frame.loc['VTemand_resp_kg'][no_vg_mean_frame.loc['VTemand_resp_kg'] < 4].sort_values()
VT_below_4_no_VG

In [ ]:
no_vg_mean_frame[VT_over_8_no_VG.index].loc['PIP'].median(), no_vg_mean_frame[VT_over_8_no_VG.index].loc['PIP'].min(), \
no_vg_mean_frame[VT_over_8_no_VG.index].loc['PIP'].max()

In [ ]:
no_vg_mean_frame[VT_below_4_no_VG.index]

In [ ]:
no_vg_mean_frame[VT_below_4_no_VG.index].loc['PIP']

#### Overall how much of all VTmand_resp_kg measurement was over 6 or 8 mL/kg

In [ ]:
VTemand_resp_all_VG = []
for case in cases_simv_vg:
    VTemand_resp_all_VG.extend(list(data_pars_measurements_vg[case]['VTemand_resp_kg']))
VTemand_resp_all_VG = Series(VTemand_resp_all_VG)

In [ ]:
VTemand_resp_all_no_VG = []
for case in cases_simv_no_vg:
    VTemand_resp_all_no_VG.extend(list(data_pars_measurements_no_vg[case]['VTemand_resp_kg']))
VTemand_resp_all_no_VG = Series(VTemand_resp_all_no_VG)

In [ ]:
len(VTemand_resp_all_VG), len(VTemand_resp_all_no_VG)

In [ ]:
VTemand_resp_all_VG.describe()

In [ ]:
VTemand_resp_all_no_VG.describe()

In [ ]:
sum(VTemand_resp_all_VG > 6), sum(VTemand_resp_all_VG > 6) / len(VTemand_resp_all_VG) * 100

In [ ]:
sum(VTemand_resp_all_VG > 8) / len(VTemand_resp_all_VG) * 100

In [ ]:
sum(VTemand_resp_all_no_VG > 6), sum(VTemand_resp_all_no_VG > 6) / len(VTemand_resp_all_no_VG) * 100

In [ ]:
sum(VTemand_resp_all_no_VG > 8) / len(VTemand_resp_all_no_VG) * 100

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([VTemand_resp_all_VG.dropna(), VTemand_resp_all_no_VG.dropna()], widths = 0.5, 
        whis = [5, 95], showfliers = False, showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 12)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('VTemand (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTemand_all_data', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

#### How many percent of the individual recordings were <4 mL/kg or >6 mL/kg

In [ ]:
above_range_VG = {}

for case in cases_simv_vg:
    a = data_pars_measurements_vg[case]['VTemand_resp_kg']
    above_range_VG[case] = [len(a), sum(a > 6), sum(a > 8)]
    
above_range_VG = DataFrame(above_range_VG).T
above_range_VG.columns = ['data_points', 'above_6', 'above_8']
above_range_VG['above_6_pc'] = above_range_VG['above_6'] / above_range_VG['data_points'] * 100
above_range_VG['above_8_pc'] = above_range_VG['above_8'] / above_range_VG['data_points'] * 100
above_range_VG.head()

In [ ]:
above_range_no_VG = {}

for case in cases_simv_no_vg:
    a = data_pars_measurements_no_vg[case]['VTemand_resp_kg']
    above_range_no_VG[case] = [len(a), sum(a > 6), sum(a > 8)]
    
above_range_no_VG = DataFrame(above_range_no_VG).T
above_range_no_VG.columns = ['data_points', 'above_6', 'above_8']
above_range_no_VG['above_6_pc'] = above_range_no_VG['above_6'] / above_range_no_VG['data_points'] * 100
above_range_no_VG['above_8_pc'] = above_range_no_VG['above_8'] / above_range_no_VG['data_points'] * 100
above_range_no_VG.head()

In [ ]:
round(above_range_VG.describe(), 1)

In [ ]:
round(above_range_no_VG.describe(), 1)

In [ ]:
stats.mannwhitneyu(above_range_VG['above_6_pc'], above_range_no_VG['above_6_pc'])

In [ ]:
stats.mannwhitneyu(above_range_VG['above_8_pc'], above_range_no_VG['above_8_pc'])

In [ ]:
fig, ax = plt.subplots(figsize = (5, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([above_range_VG['above_6_pc'], above_range_no_VG['above_6_pc']], 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(-5, 110)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('Inflations with VT > 6 mL/kg (%)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_no_VG_above_6', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = (5, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([above_range_VG['above_8_pc'], above_range_no_VG['above_8_pc']], 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(-5, 110)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('Inflations with VT > 6 mL/kg (%)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_no_VG_above_8', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
below_range_VG = {}

for case in cases_simv_vg:
    limit_vector = data_pars_measurements_vg[case]['PIP'] < data_pars_settings_vg[case]['PIP_set']
    a = data_pars_measurements_vg[case][['VTemand_resp_kg']]
    a_filtered =  a[limit_vector]
    a_filtered_2 = a[~limit_vector]
    below_range_VG[case] = [len(a), len(a_filtered), len(a_filtered_2), sum(a['VTemand_resp_kg'] < 4), 
                            sum(a['VTemand_resp_kg'] < 2), sum(a_filtered['VTemand_resp_kg'] < 4), 
                            sum(a_filtered['VTemand_resp_kg'] < 2), sum(a_filtered_2['VTemand_resp_kg'] < 4), 
                            sum(a_filtered_2['VTemand_resp_kg'] < 2)]
    
below_range_VG = DataFrame(below_range_VG).T
below_range_VG.columns = ['data_points', 'data_points_filt', 'data_points_filt_2', 
                          'below_4', 'below_2', 'below_4_filt', 'below_2_filt',
                         'below_4_filt_2', 'below_2_filt_2']
below_range_VG['below_4_pc'] = below_range_VG['below_4'] / below_range_VG['data_points'] * 100
below_range_VG['below_2_pc'] = below_range_VG['below_2'] / below_range_VG['data_points'] * 100
below_range_VG['below_4_pc_filt'] = below_range_VG['below_4_filt'] / below_range_VG['data_points_filt'] * 100
below_range_VG['below_2_pc_filt'] = below_range_VG['below_2_filt'] / below_range_VG['data_points_filt'] * 100
below_range_VG['below_4_pc_filt_2'] = below_range_VG['below_4_filt_2'] / below_range_VG['data_points_filt_2'] * 100
below_range_VG['below_2_pc_filt_2'] = below_range_VG['below_2_filt_2'] / below_range_VG['data_points_filt_2'] * 100
below_range_VG.head()

In [ ]:
below_range_VG[['data_points', 'data_points_filt', 'below_4_filt', 
                'below_2_filt','below_4_pc_filt', 'below_2_pc_filt',]].head()

In [ ]:
below_range_no_VG = {}

for case in cases_simv_no_vg:
    a = data_pars_measurements_no_vg[case]['VTemand_resp_kg']
    below_range_no_VG[case] = [len(a), sum(a < 4), sum(a < 2)]
    
below_range_no_VG = DataFrame(below_range_no_VG).T
below_range_no_VG.columns = ['data_points', 'below_4', 'below_2',]
below_range_no_VG['below_4_pc'] = below_range_no_VG['below_4'] / below_range_no_VG['data_points'] * 100
below_range_no_VG['below_2_pc'] = below_range_no_VG['below_2'] / below_range_no_VG['data_points'] * 100
below_range_no_VG.head()

In [ ]:
round(below_range_VG.describe(), 1)

In [ ]:
round(below_range_VG[['data_points', 'data_points_filt', 'below_4_filt', 
                      'below_2_filt','below_4_pc_filt', 'below_2_pc_filt',]].describe(), 1)

In [ ]:
round(below_range_no_VG.describe(), 1)

In [ ]:
stats.mannwhitneyu(below_range_VG['below_4_pc_filt'], below_range_no_VG['below_4_pc'])

In [ ]:
stats.mannwhitneyu(below_range_VG['below_4_pc'], below_range_no_VG['below_4_pc'])

In [ ]:
stats.mannwhitneyu(below_range_VG['below_2_pc_filt'], below_range_no_VG['below_2_pc'])

In [ ]:
stats.mannwhitneyu(below_range_VG['below_2_pc'], below_range_no_VG['below_2_pc'])

### Other ventilation parameters

In [ ]:
fig, ax = plt.subplots(figsize = (5, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([vg_mean_frame.loc['VTespon_pat_kg'].dropna(), no_vg_mean_frame.loc['VTespon_pat_kg'].dropna()], 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 10)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('Mean VTespon (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_noVG_VTemand_pat', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = (5, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([vg_mean_frame.loc['VTimand_kg'].dropna(), no_vg_mean_frame.loc['VTimand_kg'].dropna()], 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 18)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('Mean VTimand (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_noVG_VTimand', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = (5, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([vg_mean_frame.loc['PIP'].dropna(), no_vg_mean_frame.loc['PIP'].dropna()], 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 40)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('Mean PIP (cmH$_2$O)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_no_VG_PIP', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = (5, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([vg_mean_frame.loc['MAP'].dropna(),no_vg_mean_frame.loc['MAP'].dropna()], 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 18)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('Mean MAP (mbar)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_no_VG_MAP', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = (5, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([vg_mean_frame.loc['RR_set'].dropna(),no_vg_mean_frame.loc['RR_set'].dropna()], 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 70)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('Mean RRmand (1/min)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_no_VG_RR_set', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = (5, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([vg_mean_frame.loc['MV_kg'].dropna(), no_vg_mean_frame.loc['MV_kg'].dropna()], 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('Mean MV (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_no_VG_MV', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = (5, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([vg_mean_frame.loc['MVresp'].dropna(),no_vg_mean_frame.loc['MVresp'].dropna()], 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 100)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('Mean MVresp (%)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_no_VG_MVresp_pc', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = (5, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([vg_mean_frame.loc['FiO2_set'].dropna(),no_vg_mean_frame.loc['FiO2_set'].dropna()], 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 100)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('Mean FiO2 (%)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_no_VG_FiO2', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = (5, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([vg_mean_frame.loc['Leak'].dropna(), no_vg_mean_frame.loc['Leak'].dropna()], 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 100)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('Mean Leak (%)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_no_VG_Leak', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

## Analysing the `standard deviations (SD)` ventilation parameters of the recordings and comparing the two groups

In [ ]:
fig, ax = plt.subplots(figsize = (5, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([vg_sd_frame.loc['VTemand_resp_kg'].dropna(), no_vg_sd_frame.loc['VTemand_resp_kg'].dropna()], 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('SD VTemand (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_noVG_VTemand_resp_sd', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = (5, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([vg_sd_frame.loc['VTespon_pat_kg'].dropna(), no_vg_sd_frame.loc['VTespon_pat_kg'].dropna()], 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 10)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('SD VTespon (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_noVG_VTemand_pat_sd', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = (5, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([vg_sd_frame.loc['VTimand_kg'].dropna(), no_vg_sd_frame.loc['VTimand_kg'].dropna()], 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('SD VTimand (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_noVG_VTimand_sd', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = (5, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([vg_sd_frame.loc['PIP'].dropna(), no_vg_sd_frame.loc['PIP'].dropna()], 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('SD PIP (mbar)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_no_VG_PIP_sd', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = (5, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([vg_sd_frame.loc['MAP'].dropna(),no_vg_sd_frame.loc['MAP'].dropna()], 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('SD MAP (mbar)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_no_VG_MAP_sd', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = (5, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([vg_sd_frame.loc['RR_set'].dropna(),no_vg_sd_frame.loc['RR_set'].dropna()], 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('SD RRmand (1/min)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_no_VG_RR_set_sd', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = (5, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([vg_sd_frame.loc['MV_kg'].dropna(), no_vg_sd_frame.loc['MV_kg'].dropna()], 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 1)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('SD MV (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_no_VG_MV_sd', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = (5, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([vg_sd_frame.loc['MVresp'].dropna(), no_vg_sd_frame.loc['MVresp'].dropna()], 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 50)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('SD MVresp (%)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_no_VG_MVresp_sd', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = (5, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([vg_sd_frame.loc['FiO2_set'].dropna(),no_vg_sd_frame.loc['FiO2_set'].dropna()], 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('SD FiO2 (%)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_no_VG_FiO2_sd', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = (5, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

plt.boxplot([vg_sd_frame.loc['Leak'].dropna(), no_vg_sd_frame.loc['Leak'].dropna()], 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 50)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('SD Leak (%)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VG_no_VG_Leak_sd', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

### Save all curves for some parameters

##### VTemand_resp_kg, VG_set_kg in SIMV-VG recordings

In [ ]:
pars = ['VTemand_resp_kg', 'VG_set_kg']
name = 'VTemand_VG_set'
dim = 'mL/kg'; filetype = 'jpg'; dpi = 200

for case in cases_simv_vg:

    fig = plt.figure()
    fig.set_size_inches(8, 4)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, 
                            wspace=None, hspace=0.1)
    ax0 = fig.add_subplot(1, 1, 1)
    data_pars_measurements_vg[case][pars[0]].plot()
    data_pars_settings_vg[case][pars[1]].plot(color = 'red')

    ax0.set_xlabel('Time', size = 14, color = 'black')
    ax0.set_ylabel(dim, size = 14, color = 'black')
    ax0.set_title(case,  size = 14, color = 'black')
    ax0.legend(pars, fontsize = 12)
    ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    xticks = pd.date_range(start=data_pars_measurements_vg[case].index.min(), 
                       end=data_pars_measurements_vg[case].index.max(), freq='15min')
    # plt.xticks(xticks, xticks.strftime('%H:%M'), rotation=0, ha="center")
    ax0.tick_params(which = 'both', labelsize=14)
            
    fig.savefig('%s/%s/%s/%s_%s_%s.%s' % (DATA_DUMP, 'fabian_cases', case, case, 'SIMV_VG', name, filetype), dpi = dpi, 
        facecolor='w', edgecolor='w', orientation='portrait', 
        papertype=None, format = filetype, transparent=False, bbox_inches='tight',
        pad_inches=0.1, frameon=True);

    plt.close()

##### VTemand_resp_kg,  in SIMV-noVG recordings

In [ ]:
pars = ['VTemand_resp_kg']
name = 'VTemand'
dim = 'mL/kg'; filetype = 'jpg'; dpi = 200

for case in cases_simv_no_vg:

    fig = plt.figure()
    fig.set_size_inches(8, 4)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, 
                            wspace=None, hspace=0.1)
    ax0 = fig.add_subplot(1, 1, 1)
    data_pars_measurements_no_vg[case][pars[0]].plot()

    ax0.set_xlabel('Time', size = 14, color = 'black')
    ax0.set_ylabel(dim, size = 14, color = 'black')
    ax0.set_title(case,  size = 14, color = 'black')
    ax0.legend(pars, fontsize = 12)
    ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    xticks = pd.date_range(start=data_pars_measurements_no_vg[case].index.min(), 
                       end=data_pars_measurements_no_vg[case].index.max(), freq='15min')
    # plt.xticks(xticks, xticks.strftime('%H:%M'), rotation=0, ha="center")
    ax0.tick_params(which = 'both', labelsize=14)
            
    fig.savefig('%s/%s/%s/%s_%s_%s.%s' % (DATA_DUMP, 'fabian_cases', case, case, 'SIMV', name, filetype), dpi = dpi, 
        facecolor='w', edgecolor='w', orientation='portrait', 
        papertype=None, format = filetype, transparent=False, bbox_inches='tight',
        pad_inches=0.1, frameon=True);

    plt.close()

##### VTespon_pat in SIMV-VG recordings

In [ ]:
pars = ['VTespon_pat_kg']
name = 'VTespon_pat'
dim = 'mL/kg'; filetype = 'jpg'; dpi = 200

for case in cases_simv_vg:

    fig = plt.figure()
    fig.set_size_inches(8, 4)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, 
                            wspace=None, hspace=0.1)
    ax0 = fig.add_subplot(1, 1, 1)
    data_pars_measurements_vg[case][pars[0]].plot()

    ax0.set_xlabel('Time', size = 14, color = 'black')
    ax0.set_ylabel(dim, size = 14, color = 'black')
    ax0.set_title(case,  size = 14, color = 'black')
    ax0.legend(['VTespon_pat'], fontsize = 12)
    ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    xticks = pd.date_range(start=data_pars_measurements_vg[case].index.min(), 
                       end=data_pars_measurements_vg[case].index.max(), freq='15min')
    # plt.xticks(xticks, xticks.strftime('%H:%M'), rotation=0, ha="center")
    ax0.tick_params(which = 'both', labelsize=14)
            
    fig.savefig('%s/%s/%s/%s_%s_%s.%s' % (DATA_DUMP, 'fabian_cases', case, case, 'SIMV_VG', name, filetype), dpi = dpi, 
        facecolor='w', edgecolor='w', orientation='portrait', 
        papertype=None, format = filetype, transparent=False, bbox_inches='tight',
        pad_inches=0.1, frameon=True);

    plt.close()

##### VTespon_pat in SIMV-no-VG recordings

In [ ]:
pars = ['VTespon_pat_kg']
name = 'VTespon_pat'
dim = 'mL/kg'; filetype = 'jpg'; dpi = 200

for case in cases_simv_no_vg:

    fig = plt.figure()
    fig.set_size_inches(8, 4)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, 
                            wspace=None, hspace=0.1)
    ax0 = fig.add_subplot(1, 1, 1)
    data_pars_measurements_no_vg[case][pars[0]].plot()

    ax0.set_xlabel('Time', size = 14, color = 'black')
    ax0.set_ylabel(dim, size = 14, color = 'black')
    ax0.set_title(case,  size = 14, color = 'black')
    ax0.legend(['VTespon_pat'], fontsize = 12)
    ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    xticks = pd.date_range(start=data_pars_measurements_no_vg[case].index.min(), 
                       end=data_pars_measurements_no_vg[case].index.max(), freq='15min')
    # plt.xticks(xticks, xticks.strftime('%H:%M'), rotation=0, ha="center")
    ax0.tick_params(which = 'both', labelsize=14)
            
    fig.savefig('%s/%s/%s/%s_%s_%s.%s' % (DATA_DUMP, 'fabian_cases', case, case, 'SIMV', name, filetype), dpi = dpi, 
        facecolor='w', edgecolor='w', orientation='portrait', 
        papertype=None, format = filetype, transparent=False, bbox_inches='tight',
        pad_inches=0.1, frameon=True);

    plt.close()

##### PIP  and Pmax and alarm limits in SIMV-VG recordings

In [ ]:
pars = ['PIP', 'PIP_set', 'PIP_lim_high_set', 'PIP_lim_low_set']
name = 'PIP_with_limits'
dim = 'cmH$_2$O'; filetype = 'jpg'; dpi = 200

for case in cases_simv_vg:
    
    fig = plt.figure()
    fig.set_size_inches(8, 5)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, 
                            wspace=None, hspace=0.1)
    ax0 = fig.add_subplot(1, 1, 1)
    data_pars_measurements_vg[case][pars[0]].plot()
    data_pars_settings_vg[case][pars[1]].plot(color = 'black', linewidth = 2)
    data_pars_settings_vg[case][pars[2]].plot(color = 'red')
    data_pars_settings_vg[case][pars[3]].plot(color = 'red')

    ax0.set_xlabel('Time', size = 14, color = 'black')
    ax0.set_ylabel(dim, size = 14, color = 'black')
    ax0.set_ylim(0, 50)
    ax0.set_title(case,  size = 14, color = 'black')
    ax0.legend(['PIP', 'Pmax', 'PIP_high_alarm', 'PIP_low_alarm'], fontsize = 12)
    ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    xticks = pd.date_range(start=data_pars_measurements_vg[case].index.min(), 
                       end=data_pars_measurements_vg[case].index.max(), freq='15min')
    # plt.xticks(xticks, xticks.strftime('%H:%M'), rotation=0, ha="center")
    ax0.tick_params(which = 'both', labelsize=12)
            
    fig.savefig('%s/%s/%s/%s_%s_%s.%s' % (DATA_DUMP, 'fabian_cases', case, case, 'SIMV-VG', name, filetype), dpi = dpi, 
        facecolor='w', edgecolor='w', orientation='portrait', 
        papertype=None, format = filetype, transparent=False, bbox_inches='tight',
        pad_inches=0.1, frameon=True);

    plt.close()

##### PIP and alarm limits in SIMV-no-VG recordings

In [ ]:
pars = ['PIP', 'PIP_lim_high_set', 'PIP_lim_low_set']
name = 'PIP'
dim = 'cmH$_2$O'; filetype = 'jpg'; dpi = 200

for case in cases_simv_no_vg:
    
    fig = plt.figure()
    fig.set_size_inches(8, 5)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, 
                            wspace=None, hspace=0.1)
    ax0 = fig.add_subplot(1, 1, 1)
    data_pars_measurements_no_vg[case][pars[0]].plot()
    data_pars_settings_no_vg[case][pars[1]].plot(color = 'red')
    data_pars_settings_no_vg[case][pars[2]].plot(color = 'red')

    ax0.set_xlabel('Time', size = 14, color = 'black')
    ax0.set_ylabel(dim, size = 14, color = 'black')
    ax0.set_ylim(0, 50)
    ax0.set_title(case,  size = 14, color = 'black')
    ax0.legend(['PIP', 'Pmax', 'PIP_high_alarm', 'PIP_low_alarm'], fontsize = 12)
    ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    xticks = pd.date_range(start=data_pars_measurements_no_vg[case].index.min(), 
                       end=data_pars_measurements_no_vg[case].index.max(), freq='15min')
    # plt.xticks(xticks, xticks.strftime('%H:%M'), rotation=0, ha="center")
    ax0.tick_params(which = 'both', labelsize=12)
            
    fig.savefig('%s/%s/%s/%s_%s_%s.%s' % (DATA_DUMP, 'fabian_cases', case, case, 'SIMV', name, filetype), dpi = dpi, 
        facecolor='w', edgecolor='w', orientation='portrait', 
        papertype=None, format = filetype, transparent=False, bbox_inches='tight',
        pad_inches=0.1, frameon=True);

    plt.close()

##### MV in SIMV-VG recordings

In [ ]:
pars = ['MV_kg']
name = 'MV'
dim = 'mL/kg'; filetype = 'jpg'; dpi = 200

for case in cases_simv_vg:
    
    fig = plt.figure()
    fig.set_size_inches(8, 5)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, 
                            wspace=None, hspace=0.1)
    ax0 = fig.add_subplot(1, 1, 1)
    data_pars_measurements_vg[case][pars[0]].plot(color = 'blue')
    
    ax0.set_xlabel('Time', size = 14, color = 'black')
    ax0.set_ylabel(dim, size = 14, color = 'black')
    ax0.set_ylim(0, 1)
    ax0.set_title(case,  size = 14, color = 'black')
    ax0.legend(pars, fontsize = 12)
    ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    xticks = pd.date_range(start=data_pars_measurements_vg[case].index.min(), 
                       end=data_pars_measurements_vg[case].index.max(), freq='15min')
    # plt.xticks(xticks, xticks.strftime('%H:%M'), rotation=0, ha="center")
    ax0.tick_params(which = 'both', labelsize=12)
            
    fig.savefig('%s/%s/%s/%s_%s_%s.%s' % (DATA_DUMP, 'fabian_cases', case, case, 'SIMV-VG', name, filetype), dpi = dpi, 
        facecolor='w', edgecolor='w', orientation='portrait', 
        papertype=None, format = filetype, transparent=False, bbox_inches='tight',
        pad_inches=0.1, frameon=True);

    plt.close()

##### MV in SIMV-no-VG recordings

In [ ]:
pars = ['MV_kg']
name = 'MV'
dim = 'mL/kg'; filetype = 'jpg'; dpi = 200

for case in cases_simv_no_vg:
    
    fig = plt.figure()
    fig.set_size_inches(8, 5)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, 
                            wspace=None, hspace=0.1)
    ax0 = fig.add_subplot(1, 1, 1)
    data_pars_measurements_no_vg[case][pars[0]].plot(color = 'blue')
    
    ax0.set_xlabel('Time', size = 14, color = 'black')
    ax0.set_ylabel(dim, size = 14, color = 'black')
    ax0.set_ylim(0, 1)
    ax0.set_title(case,  size = 14, color = 'black')
    ax0.legend(pars, fontsize = 12)
    ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    xticks = pd.date_range(start=data_pars_measurements_no_vg[case].index.min(), 
                       end=data_pars_measurements_no_vg[case].index.max(), freq='15min')
    # plt.xticks(xticks, xticks.strftime('%H:%M'), rotation=0, ha="center")
    ax0.tick_params(which = 'both', labelsize=12)
            
    fig.savefig('%s/%s/%s/%s_%s_%s.%s' % (DATA_DUMP, 'fabian_cases', case, case, 'SIMV', name, filetype), dpi = dpi, 
        facecolor='w', edgecolor='w', orientation='portrait', 
        papertype=None, format = filetype, transparent=False, bbox_inches='tight',
        pad_inches=0.1, frameon=True);

    plt.close()

##### MVresp (%) in SIMV-VG recordings

In [ ]:
pars = ['MVresp']
name = 'MVresp'
dim = '%'; filetype = 'jpg'; dpi = 200

for case in cases_simv_vg:
    
    fig = plt.figure()
    fig.set_size_inches(8, 5)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, 
                            wspace=None, hspace=0.1)
    ax0 = fig.add_subplot(1, 1, 1)
    data_pars_measurements_vg[case][pars[0]].plot(color = 'blue')
    
    ax0.set_xlabel('Time', size = 14, color = 'black')
    ax0.set_ylabel(dim, size = 14, color = 'black')
    ax0.set_ylim(0, 105)
    ax0.set_title(case,  size = 14, color = 'black')
    ax0.legend(pars, fontsize = 12)
    ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    xticks = pd.date_range(start=data_pars_measurements_vg[case].index.min(), 
                       end=data_pars_measurements_vg[case].index.max(), freq='15min')
    # plt.xticks(xticks, xticks.strftime('%H:%M'), rotation=0, ha="center")
    ax0.tick_params(which = 'both', labelsize=12)
            
    fig.savefig('%s/%s/%s/%s_%s_%s.%s' % (DATA_DUMP, 'fabian_cases', case, case, 'SIMV_VG', name, filetype), dpi = dpi, 
        facecolor='w', edgecolor='w', orientation='portrait', 
        papertype=None, format = filetype, transparent=False, bbox_inches='tight',
        pad_inches=0.1, frameon=True);

    plt.close()

##### MVresp (%) in SIMV-no-VG recordings

In [ ]:
pars = ['MVresp']
name = 'MVresp'
dim = '%'; filetype = 'jpg'; dpi = 200

for case in cases_simv_no_vg:
    
    fig = plt.figure()
    fig.set_size_inches(8, 5)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, 
                            wspace=None, hspace=0.1)
    ax0 = fig.add_subplot(1, 1, 1)
    data_pars_measurements_no_vg[case][pars[0]].plot(color = 'blue')
    
    ax0.set_xlabel('Time', size = 14, color = 'black')
    ax0.set_ylabel(dim, size = 14, color = 'black')
    ax0.set_ylim(0, 105)
    ax0.set_title(case,  size = 14, color = 'black')
    ax0.legend(pars, fontsize = 12)
    ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    xticks = pd.date_range(start=data_pars_measurements_no_vg[case].index.min(), 
                       end=data_pars_measurements_no_vg[case].index.max(), freq='15min')
    # plt.xticks(xticks, xticks.strftime('%H:%M'), rotation=0, ha="center")
    ax0.tick_params(which = 'both', labelsize=12)
            
    fig.savefig('%s/%s/%s/%s_%s_%s.%s' % (DATA_DUMP, 'fabian_cases', case, case, 'SIMV', name, filetype), dpi = dpi, 
        facecolor='w', edgecolor='w', orientation='portrait', 
        papertype=None, format = filetype, transparent=False, bbox_inches='tight',
        pad_inches=0.1, frameon=True);

    plt.close()

## pCO2 before and after transport

In [ ]:
blood_gases_simv_vg = {case: blood_gases[case] for case in blood_gases.keys()
                       if case in cases_simv_vg }
len(blood_gases_simv_vg)

In [ ]:
blood_gases_simv_no_vg = {case: blood_gases[case] for case in blood_gases.keys()
                       if case in cases_simv_no_vg }
len(blood_gases_simv_no_vg)

In [ ]:
CO2_before = {}
CO2_after = {}
for case in blood_gases_simv_vg:
    if len(blood_gases_simv_vg[case].index) > 1:
        CO2_before[case] = float(blood_gases_simv_vg[case].iloc[0]['pCO2'])
        CO2_after[case] = float(blood_gases_simv_vg[case].iloc[-1]['pCO2'])
    
CO2_simv_vg = DataFrame([CO2_before, CO2_after]).T
CO2_simv_vg.columns = ['before', 'after']
CO2_simv_vg.dropna(how = 'any', inplace = True)
CO2_simv_vg['diff'] = CO2_simv_vg['before'] - CO2_simv_vg['after']

In [ ]:
CO2_simv_vg.info()

In [ ]:
CO2_simv_vg.head()

In [ ]:
CO2_before = {}
CO2_after = {}
for case in blood_gases_simv_no_vg:
    if len(blood_gases_simv_no_vg[case].index) > 1:
        CO2_before[case] = float(blood_gases_simv_no_vg[case].iloc[0]['pCO2'])
        CO2_after[case] = float(blood_gases_simv_no_vg[case].iloc[-1]['pCO2'])
    
CO2_simv_no_vg = DataFrame([CO2_before, CO2_after]).T
CO2_simv_no_vg.columns = ['before', 'after']
CO2_simv_no_vg.dropna(how = 'any', inplace = True)
CO2_simv_no_vg['diff'] = CO2_simv_no_vg['before'] - CO2_simv_no_vg['after']

In [ ]:
CO2_simv_no_vg.info()

In [ ]:
CO2_simv_no_vg.head()

In [ ]:
CO2_simv_vg['after'].describe()

In [ ]:
CO2_simv_no_vg['after'].describe()

In [ ]:
# Arrival blood gases with < 35 mmHg (5 kPa)
CO2_simv_vg['after'][CO2_simv_vg['after'] < 37.5] 

In [ ]:
# Arrival blood gases with < 35 mmHg (5 kPa)
CO2_simv_no_vg['after'][CO2_simv_no_vg['after'] < 37.5] 

In [ ]:
# Arrival blood gases with < 35 mmHg (4 kPa)
CO2_simv_vg['after'][CO2_simv_vg['after'] < 30] 

In [ ]:
# Arrival blood gases with < 35 mmHg (4 kPa)
CO2_simv_no_vg['after'][CO2_simv_no_vg['after'] < 30] 

In [ ]:
CO2_simv_vg['diff'].describe()

In [ ]:
CO2_simv_no_vg['diff'].describe()

In [ ]:
blood_gases['AL000038']

In [ ]:
blood_gases['AL000120']

In [ ]:
for rec in VT_below_4_VG.index:
    print(rec, blood_gases[rec].iloc[-1]['pCO2'])

In [ ]:
for rec in VT_below_4_no_VG.index:
    print(rec, blood_gases[rec].iloc[-1]['pCO2'])

In [ ]:
for rec in VT_over_6_VG.index:
    print(rec, blood_gases[rec].iloc[-1]['pCO2'])

In [ ]:
for rec in VT_over_6_no_VG.index:
    try:
        print(rec, blood_gases[rec].iloc[-1]['pCO2'])
    except KeyError:
        print(rec, 'no blood gas')

In [ ]:
for rec in VT_over_8_no_VG.index:
    try:
        print(rec, blood_gases[rec].iloc[-1]['pCO2'])
    except KeyError:
        print(rec, 'no blood gas')

In [ ]:
stats.ttest_ind(CO2_simv_vg['after'], CO2_simv_no_vg['after'])

## Tables and Figure for the paper

### Table 1

In [ ]:
clin_df_stats_VG_no_VG

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'Table_1.xlsx'))
clin_df_stats_VG_no_VG.to_excel(writer, 'clinical_details')
writer.save()

### Supplementary Table 1

In [ ]:
stats_pars_all_vg_frame_2.head()

In [ ]:
column_order = ['FiO2_set', 'PIP_set', 'PIP', 'MAP',  'PEEP_set', 'PEEP', 'VG_set_kg',
                'VTemand_kg', 'VTemand_resp_kg', 'VTespon_pat_kg', 'VTimand_kg', 
                 'RR_set',  'Ti_set', 'MV_kg', 'MVresp', 'Leak',] 
    

Supplementary_Table_1 = stats_pars_all_vg_frame_2[column_order]

In [ ]:
Supplementary_Table_1.head()

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'Supplementary_Table_1.xlsx'))
Supplementary_Table_1.to_excel(writer, 'stats_VG')
writer.save()

### Supplementary_Table_2

In [ ]:
stats_pars_all_no_vg_frame_2.head()

In [ ]:
column_order = ['FiO2_set', 'PIP_set', 'PIP', 'MAP',  'PEEP_set', 'PEEP', 
                'VTemand_kg', 'VTemand_resp_kg', 'VTespon_pat_kg', 'VTimand_kg', 
                 'RR_set',  'Ti_set', 'MV_kg', 'MVresp', 'Leak',] 
    

Supplementary_Table_2 = stats_pars_all_no_vg_frame_2[column_order]

In [ ]:
Supplementary_Table_2.head()

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'Supplementary_Table_2.xlsx'))
Supplementary_Table_2.to_excel(writer, 'stats_no_VG')
writer.save()

### Table 2

In [ ]:
VG_stats_mean.sort_values('corrected p')

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'Table_2.xlsx'))
VG_stats_mean.sort_values('corrected p').to_excel(writer, 'VG_stats_mean')
writer.save()

### Table 3

In [ ]:
VG_stats_sd.sort_values('corrected p')

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'Table_3.xlsx'))
VG_stats_sd.sort_values('corrected p').to_excel(writer, 'VG_stats_sd')
writer.save()

### Figure 2

#### Figure 2A

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}
meanprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}

plt.boxplot([VTemand_resp_all_VG.dropna(), VTemand_resp_all_no_VG.dropna()], widths = 0.5, 
        whis = [5, 95], showfliers = False, showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops, meanprops = meanprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(0, 12)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('VTe (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2A', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

#### Figure 2B

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}
meanprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}

plt.boxplot([above_range_VG['above_6_pc'], above_range_no_VG['above_6_pc']], widths = 0.5, 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops, meanprops = meanprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(-5, 110)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('VTe > 6 mL/kg (%)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2B', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

#### Figure 2C

In [ ]:
fig, ax = plt.subplots(figsize = (3, 4))
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}
meanprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}

plt.boxplot([above_range_VG['above_8_pc'], above_range_no_VG['above_8_pc']], widths = 0.5, 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops, meanprops = meanprops)

ax.set_xticklabels(xticklabels)
ax.set_ylim(-5, 110)
ax.set_xlabel('Ventilation', size = 14)
ax.set_ylabel('VTe > 8 mL/kg (%)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2C', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

#### Figure 2 combined

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['VG', 'no VG']

fig, ax = plt.subplots(1,3, figsize = (12, 4))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, hspace=0, wspace=0.5)

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}
meanprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}

# Figure 2A
ax[0].boxplot([VTemand_resp_all_VG.dropna(), VTemand_resp_all_no_VG.dropna()], widths = 0.5, 
    whis = [5, 95], showfliers = False, showmeans = True, medianprops=medianprops, boxprops=boxprops, 
    whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops, meanprops = meanprops)

ax[0].set_xticklabels(xticklabels)
ax[0].set_ylim(0, 14)
ax[0].set_xlabel('Ventilation', size = 14)
ax[0].set_ylabel('VTemand (mL/kg)', size = 14)
ax[0].tick_params(axis='both', which='major', labelsize=14)
# ax[0].grid(True)

# Figure 2B
ax[1].boxplot([above_range_VG['above_6_pc'], above_range_no_VG['above_6_pc']], widths = 0.5, 
        whis = [5, 95], showfliers = True, showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops, meanprops = meanprops)

ax[1].set_xticklabels(xticklabels)
ax[1].set_ylim(-5, 140)
ax[1].set_xlabel('Ventilation', size = 14)
ax[1].set_ylabel('VTemand > 6 mL/kg (%)', size = 14)
ax[1].tick_params(axis='both', which='major', labelsize=14)
# ax[1].grid(True)

# Figure 2C
ax[2].boxplot([above_range_VG['above_8_pc'], above_range_no_VG['above_8_pc']], widths = 0.5, 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops, meanprops = meanprops)

ax[2].set_xticklabels(xticklabels)
ax[2].set_ylim(-5, 140)
ax[2].set_xlabel('Ventilation', size = 14)
ax[2].set_ylabel('VTemand > 8 mL/kg (%)', size = 14)
ax[2].tick_params(axis='both', which='major', labelsize=14)
# ax[2].grid(True)

fig.text(0.06, 0.96, 'A', fontsize = 14); fig.text(0.34, 0.96, 'B', fontsize = 14)
fig.text(0.64, 0.96, 'C', fontsize = 14)

fig.text(0.14, 0.57, 'n=106,765', fontsize = 12); fig.text(0.24, 0.75, 'n=56,032', fontsize = 12)
fig.text(0.44, 0.72, 'n=45', fontsize = 12); fig.text(0.54, 0.72, 'n=32', fontsize = 12)
fig.text(0.73, 0.42, 'n=45', fontsize = 12); fig.text(0.83, 0.72, 'n=32', fontsize = 12)


fig.text(0.48, 0.83, 'p<0.0001', fontsize = 12); fig.text(0.76, 0.83, 'p<0.0001', fontsize = 12)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

In [ ]:
dpi = 400
filetype = 'tiff'
xticklabels = ['VG', 'no VG']

fig, ax = plt.subplots(1,3, figsize = (12, 4))
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, hspace=0, wspace=0.5)

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}
meanprops = {'marker':'D', 'markeredgecolor':'black', 'markerfacecolor':'black'}

# Figure 2A
ax[0].boxplot([VTemand_resp_all_VG.dropna(), VTemand_resp_all_no_VG.dropna()], widths = 0.5, 
    whis = [5, 95], showfliers = False, showmeans = True, medianprops=medianprops, boxprops=boxprops, 
    whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops, meanprops = meanprops)

ax[0].set_xticklabels(xticklabels)
ax[0].set_ylim(0, 14)
ax[0].set_xlabel('Ventilation', size = 14)
ax[0].set_ylabel('VTemand (mL/kg)', size = 14)
ax[0].tick_params(axis='both', which='major', labelsize=14)
# ax[0].grid(True)

# Figure 2B
ax[1].boxplot([above_range_VG['above_6_pc'], above_range_no_VG['above_6_pc']], widths = 0.5, 
        whis = [5, 95], showfliers = True, showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops, meanprops = meanprops)

ax[1].set_xticklabels(xticklabels)
ax[1].set_ylim(-5, 140)
ax[1].set_xlabel('Ventilation', size = 14)
ax[1].set_ylabel('VTemand > 6 mL/kg (%)', size = 14)
ax[1].tick_params(axis='both', which='major', labelsize=14)
# ax[1].grid(True)

# Figure 2C
ax[2].boxplot([above_range_VG['above_8_pc'], above_range_no_VG['above_8_pc']], widths = 0.5, 
        whis = [5, 95], showfliers = True,showmeans = True, medianprops=medianprops, boxprops=boxprops, 
        whiskerprops=whiskerprops, capprops=capprops, flierprops = flierprops, meanprops = meanprops)

ax[2].set_xticklabels(xticklabels)
ax[2].set_ylim(-5, 140)
ax[2].set_xlabel('Ventilation', size = 14)
ax[2].set_ylabel('VTemand > 8 mL/kg (%)', size = 14)
ax[2].tick_params(axis='both', which='major', labelsize=14)
# ax[2].grid(True)

fig.text(0.06, 0.96, 'A', fontsize = 14); fig.text(0.34, 0.96, 'B', fontsize = 14)
fig.text(0.64, 0.96, 'C', fontsize = 14)

fig.text(0.14, 0.57, 'n=106,765', fontsize = 12); fig.text(0.24, 0.75, 'n=56,032', fontsize = 12)
fig.text(0.44, 0.72, 'n=45', fontsize = 12); fig.text(0.54, 0.72, 'n=32', fontsize = 12)
fig.text(0.73, 0.42, 'n=45', fontsize = 12); fig.text(0.83, 0.72, 'n=32', fontsize = 12)


fig.text(0.48, 0.83, 'p<0.0001', fontsize = 12); fig.text(0.76, 0.83, 'p<0.0001', fontsize = 12)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=True);

### Supplementary Figure 1

#### Supplementary Figure 1A

In [ ]:
pars = ['VTemand_resp_kg']
name = 'VTemand'
dim = 'mL/kg'; filetype = 'jpg'; dpi = 200

for case in ['AL000126']:

    fig = plt.figure()
    fig.set_size_inches(8, 4)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, 
                            wspace=None, hspace=0.1)
    ax0 = fig.add_subplot(1, 1, 1)
    data_pars_measurements_no_vg[case][pars[0]].plot()

    ax0.set_xlabel('Time', size = 14, color = 'black')
    ax0.set_ylabel(dim, size = 14, color = 'black')
    ax0.set_title('',  size = 14, color = 'black')
    ax0.legend(['VTe'], fontsize = 12)
    ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    xticks = pd.date_range(start=data_pars_measurements_no_vg[case].index.min(), 
                       end=data_pars_measurements_no_vg[case].index.max(), freq='15min')
    ax0.tick_params(which = 'both', labelsize=14)
            
    fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Supplementary_Figure_1A', filetype), dpi = dpi, 
        facecolor='w', edgecolor='w', orientation='portrait', 
        papertype=None, format = filetype, transparent=False, bbox_inches='tight',
        pad_inches=0.1, frameon=True);

#### FigureSupplementary Figure 1B

In [ ]:
pars = ['PIP', 'PIP_set', ]
pars_2 = ['PIP', 'PIP set']

dim = 'cmH$_2$O'; filetype = 'jpg'; dpi = 200

for case in ['AL000126']:
    
    fig = plt.figure()
    fig.set_size_inches(8, 3)
    fig.subplots_adjust(left=None, bottom=None, right=None, top=None, 
                            wspace=None, hspace=0.1)
    ax0 = fig.add_subplot(1, 1, 1)
    data_pars_measurements_no_vg[case][pars[0]].plot(color = 'red')
    data_pars_settings_no_vg[case][pars[1]].plot(color = 'black', linewidth = 2)
    
    ax0.set_xlabel('Time', size = 14, color = 'black')
    ax0.set_ylabel(dim, size = 14, color = 'black')
    ax0.set_ylim(0, 30)
    ax0.set_title('',  size = 14, color = 'black')
    ax0.legend(pars, fontsize = 12)
    ax0.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
    xticks = pd.date_range(start=data_pars_measurements_no_vg[case].index.min(), 
                       end=data_pars_measurements_no_vg[case].index.max(), freq='15min')
    ax0.tick_params(which = 'both', labelsize=14)
            
    fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Supplementary_Figure_1B', filetype), dpi = dpi, 
        facecolor='w', edgecolor='w', orientation='portrait', 
        papertype=None, format = filetype, transparent=False, bbox_inches='tight',
        pad_inches=0.1, frameon=True);

#### Supplementary Figure 1 combined

In [ ]:
dpi = 300
filetype = 'jpg'

fig, ax = plt.subplots(2, 1, figsize = [8,8])
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,  wspace=None, hspace=0.1)

# Supplementary Figure 1A
data_pars_measurements_no_vg['AL000126']['VTemand_resp_kg'].plot(ax = ax[0],)
ax[0].get_xaxis().set_visible(False)
#plt.xaxis('off')

ax[0].set_ylabel('(mL/kg)', size = 14, color = 'black')
ax[0].set_title('',  size = 14, color = 'black')
ax[0].legend(['VTemand'], fontsize = 14, loc = 4)
ax[0].grid(True, linestyle='-', linewidth=0.5, color = 'gray')
ax[0].tick_params(which = 'both', labelsize=14)

# Supplementary Figure 1B
data_pars_measurements_no_vg['AL000126']['PIP'].plot(ax = ax[1], color = 'red',)
data_pars_settings_no_vg['AL000126']['PIP_set'].plot(ax = ax[1], color = 'black', linewidth = 2)
ax[1].set_xlabel('Time', size = 14, color = 'black')
ax[1].set_ylabel('cmH$_2$O', size = 14, color = 'black')
ax[1].set_ylim(0, 30)
ax[1].set_title('',  size = 14, color = 'black')
ax[1].legend(['PIP', 'PIPset'], fontsize = 14)
ax[1].grid(True, linestyle='-', linewidth=0.5, color = 'gray')
ax[1].tick_params(which = 'both', labelsize=14)
            
fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Supplementary_Figure_1', filetype), dpi = dpi, 
        facecolor='w', edgecolor='w', orientation='portrait', 
        papertype=None, format = filetype, transparent=False, bbox_inches='tight',
        pad_inches=0.1, frameon=True);

In [ ]:
dpi = 400
filetype = 'tiff'

fig, ax = plt.subplots(2, 1, figsize = [6,6])
fig.subplots_adjust(left=None, bottom=None, right=None, top=None,  wspace=None, hspace=0.1)

# Supplementary Figure 1A
data_pars_measurements_no_vg['AL000126']['VTemand_resp_kg'].plot(ax = ax[0],)
ax[0].get_xaxis().set_visible(False)
#plt.xaxis('off')

ax[0].set_ylabel('(mL/kg)', size = 14, color = 'black')
ax[0].set_title('',  size = 14, color = 'black')
ax[0].legend(['VTemand'], fontsize = 14, loc = 4)
ax[0].grid(True, linestyle='-', linewidth=0.5, color = 'gray')
ax[0].tick_params(which = 'both', labelsize=14)

# Supplementary Figure 1B
data_pars_measurements_no_vg['AL000126']['PIP'].plot(ax = ax[1], color = 'red',)
data_pars_settings_no_vg['AL000126']['PIP_set'].plot(ax = ax[1], color = 'black', linewidth = 2)
ax[1].set_xlabel('Time', size = 14, color = 'black')
ax[1].set_ylabel('cmH$_2$O', size = 14, color = 'black')
ax[1].set_ylim(0, 30)
ax[1].set_title('',  size = 14, color = 'black')
ax[1].legend(['PIP', 'PIPset'], fontsize = 14)
ax[1].grid(True, linestyle='-', linewidth=0.5, color = 'gray')
ax[1].tick_params(which = 'both', labelsize=14)
            
fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Supplementary_Figure_1', filetype), dpi = dpi, 
        facecolor='w', edgecolor='w', orientation='portrait', 
        papertype=None, format = filetype, transparent=False, bbox_inches='tight',
        pad_inches=0.1, frameon=True);